# Модуль shelve (из стандартного набора Python) 

Данный модуль позволяет организовать постоянное хранилище информации без прибегания к реляционным базам данных. Доступ к хранилищу shelve осуществляется с помощью ключей, как при работе с обычными словарями. Модуль shelve сериализует объект с помощью модуля pickle и записывает его в БД, создаваемую и управляемую с помощью dbm.

In [57]:
import shelve

with shelve.open('test_shelf.db') as s:
    s['key1']={
        'int': 10,
        'float': 9.5,
        'string' : 'Sample data'} 

Проще всего использовать модуль shelve посредством класса Dbfilename
Shelf. В свою очередь для сохранения данных этот класс использует интерфейс
dbm. Класс DbfilenameShelf можно использовать либо непосредственно, либо путем вызова функции shelve. open ().

In [52]:
import shelve

with shelve.open('test_shelf.db') as s:
    existing = s['key1']
print(existing)


{'int': 10, 'float': 9.5, 'string': 'Sample data'}


Модуль dbm не поддерживает одновременную запись в одну и ту же базу данных несколькими приложениями одновременно, но поддерживает параллельные обращения к ней по чтению несколькими клиентами. Если клиент не будет изменять хранилище, откройте базу данных только для чтения, передав аргумент flag='r'.

In [53]:
import dbm
import shelve

with shelve.open('test_shelf.db', flag='r') as s:
    print('Existing:', s['key1'])
    try:
        s['key1']= 'new value'
    except dbm.error as err:
        print('ERROR: {}'.format(err))

Existing: {'int': 10, 'float': 9.5, 'string': 'Sample data'}
ERROR: The database is opened for reading only


Если программа попытается изменить базу данных, в то время как она открыта только для чтения, будет сгенерировано исключение.Тип исключения зависит от того, какой модуль базы данных был выбран модулем dbm при ее создании.

Кроме флага r(только для чтения), также существуют ещё три флага:

c: файл открывается для чтения и записи (значение по умолчанию). Если файл не существует, то он создается.

w: файл открывается для записи.

n: файл открывается для записи Если файл не существует, то он создается. Если он существует, то он перезаписывается

# Обратная запись

По умолчанию хранилища, создаваемые модулем shelve, не отслеживают изменения, происходящие во временных объектах. Таким образом, если содержимое объекта, ранее сохраненного в хранилище, изменяется, хранилище необходимо обновить явно путем повторного сохранения в нем данного элемента.

In [55]:
import shelve

with shelve.open('test_shelf.db') as s:
    print(s['key1'])
    s['key1']['new_value'] = 'this was not here before'
    
with shelve.open('test_shelf.db', writeback = True) as s:
    print(s['key1'])

{'int': 10, 'float': 9.5, 'string': 'Sample data'}
{'int': 10, 'float': 9.5, 'string': 'Sample data'}


В этом примере значение, соответствующее ключу ’ key1', не сохраняется повторно, в чем можно убедиться, повторно открыв хранилище.

Чтобы обеспечить автоматическое обновление хранилища при изменении объектов, временно существующих в памяти компьютера, его необходимо открыть c установленным флагом writeback. Установка этого флага приводит к тому, что хранилище будет запоминать в кеш-памяти все объекты, извлеченные из базы данных. При закрытии хранилища каждый объект, хранящийся в кеше, записывается в базу данных.

In [56]:
import shelve
import pprint
with shelve.open('test_shelf.db', writeback=True) as s:
    print('Initial data: ')
    pprint.pprint(s['key1'])
    
    s['key1']['new_value']='this was not here before'
    print('\nModified: ')
    pprint.pprint(s['key1'])
with shelve.open('test_shelf.db', writeback=True) as s:
    print('n\Preserved:')
    pprint.pprint(s['key1'])

Initial data: 
{'float': 9.5, 'int': 10, 'string': 'Sample data'}

Modified: 
{'float': 9.5,
 'int': 10,
 'new_value': 'this was not here before',
 'string': 'Sample data'}
n\Preserved:
{'float': 9.5,
 'int': 10,
 'new_value': 'this was not here before',
 'string': 'Sample data'}


Несмотря на то что указанная мера позволяет снизить вероятность совершения ошибок программистом и может сделать организацию долговременного хранения объектов более прозрачной, использование режима обратной записи не всегда желательно. Создание кеша при открытии хранилища приводит к дополнительному расходованию памяти и приостанавливает выполнение приложения всякий раз, когда кешированный объект записывается в базу данных. В нее записываются все кешированные объекты, поскольку нет возможности явно указать, какие именно объекты претерпели изменения. Если приложение в основном читает, а не записывает данные, режим обратной записи может ухудшить его производительность.

# Получение записей из БД 

Также из БД можно получать информацию по ключам, значениям и целым кортежам. Для этого используются методы keys(), values() и items(). Для этого создадим БД

In [1]:
import shelve
 
FILENAME = "states2"
with shelve.open(FILENAME) as states:
    states["London"] = "Great Britain"
    states["Paris"] = "France"
    states["Berlin"] = "Germany"
    states["Madrid"] = "Spain"
 
with shelve.open(FILENAME) as states:
    print(states["London"])
    print(states["Madrid"])

Great Britain
Spain


Метод keys() возвращает все ключи из файла, а метод values() - все значения:

In [2]:
with shelve.open(FILENAME) as states:
 
    for city in states.keys():
        print(city, end=" ")        # London Paris Berlin Madrid 
    print()
    for country in states.values():
        print(country, end=" ")     # Great Britain France Germany Spain 

London Paris Berlin Madrid 
Great Britain France Germany Spain 

Еще один метод items() возвращает набор кортежей. Каждый кортеж содержит ключ и значение.

In [3]:
with shelve.open(FILENAME) as states:
 
    for state in states.items():
        print(state)

('London', 'Great Britain')
('Paris', 'France')
('Berlin', 'Germany')
('Madrid', 'Spain')


# Домашнее задание

Сделать несколько записей (кортежей) и вернуть их с помощью метода items()